In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math

In [2]:
df = pd.read_csv("players_21.csv")

In [3]:
typ_dict= dict(df.dtypes)
url_col = list()
for typ in typ_dict:
  if typ_dict[typ] == object and "url" in typ:
    url_col.append(typ)
df.drop(url_col, axis=1, inplace=True)

In [4]:
url_col

['player_url',
 'player_face_url',
 'club_logo_url',
 'club_flag_url',
 'nation_logo_url',
 'nation_flag_url']

In [5]:
null_df = dict(df.isnull().sum())
null_col = list()
for col in null_df:
    if null_df[col] > 9000:
        null_col.append(col)
df.drop(null_col, axis=1, inplace=True)

In [6]:
null_col

['club_loaned_from',
 'nation_team_id',
 'nation_position',
 'nation_jersey_number',
 'player_tags',
 'player_traits',
 'goalkeeping_speed']

In [7]:
typ_dict= dict(df.dtypes)
obj_list = list()
for typ in typ_dict:
  if typ_dict[typ] == object:
    obj_list.append(typ)

In [8]:
#converting columns with dtype object but supposed to be int; all because the values are in the str format of "89+3"; I have discarded the +/1
for col in obj_list[13:]:
    tmp = df[col].tolist()
    tmp_ = [x.split("+") for x in tmp]
    tmp_ = [re.split("[+-]", x)[0] for x in tmp]
    #tmp_ = [[int(y) for y in x] for x in tmp_]
    df[col] = tmp_
df.drop(obj_list[:13], axis=1, inplace=True)

In [9]:
l = list(df.columns)
l.remove("league_level")
df.dropna(axis=0, inplace=True, subset=l)

In [10]:
df.dropna(axis=1, how="any", inplace=True)

In [11]:
df

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023,93,93,103500000.0,560000.0,33,170,72,241.0,1.0,...,65,65,65,66,62,52,52,52,62,19
1,20801,92,92,63000000.0,220000.0,35,187,83,45.0,1.0,...,61,61,61,65,61,54,54,54,61,20
2,188545,91,91,111000000.0,240000.0,31,184,80,21.0,1.0,...,65,65,65,64,61,60,60,60,61,19
3,190871,91,91,132000000.0,270000.0,28,175,68,73.0,1.0,...,62,62,62,67,62,49,49,49,62,20
4,192985,91,91,129000000.0,370000.0,29,181,70,10.0,1.0,...,80,80,80,79,75,69,69,69,75,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,257710,47,52,70000.0,1000.0,21,177,70,112165.0,1.0,...,41,41,41,42,45,47,47,47,45,15
18940,257933,47,53,70000.0,1000.0,21,174,68,112540.0,1.0,...,48,48,48,47,47,46,46,46,47,15
18941,257936,47,47,45000.0,2000.0,28,185,79,111774.0,1.0,...,47,47,47,47,47,46,46,46,47,11
18942,258736,47,67,130000.0,500.0,17,171,58,1920.0,4.0,...,32,32,32,35,33,26,26,26,33,14


In [12]:
df = df.astype('int')

X = df.drop('league_level', axis=1)
y = df['league_level']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [13]:
fwd=["ls","st","rs","lw","lf","cf","rw","rf"]
mid=["lam","cam","ram","lm","lcm","cm","rcm","rm","ldm","cdm","rdm"]
bck=["lwb","rwb","lb","lcb","cb","rcb","rb","gk"]

In [14]:
# sc = StandardScaler()

# tr_col = list(X_train.columns)
# te_col = list(X_test.columns)

# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

# X_train = pd.DataFrame(X_train, columns=tr_col)
# X_test= pd.DataFrame(X_test, columns=te_col)


In [15]:
pca = PCA(n_components=1)
X_train["fwd"] = pca.fit_transform(X_train[fwd])
X_test["fwd"] = pca.transform(X_test[fwd])

X_train["mid"] = pca.fit_transform(X_train[mid])
X_test["mid"] = pca.transform(X_test[mid])

X_train["bck"] = pca.fit_transform(X_train[bck])
X_test["bck"] = pca.transform(X_test[bck])

In [16]:
X_train.drop(labels=fwd+mid+bck, axis=1, inplace=True)
X_test.drop(labels=fwd+mid+bck, axis=1, inplace=True)

In [17]:
lr_model = Lasso(alpha=0.1)
lr_model.fit(X_train, y_train)

print('Training score: {}'.format(lr_model.score(X_train, y_train)))
print('Test score: {}'.format(lr_model.score(X_test, y_test)))

y_pred = lr_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)

print('RMSE: {}'.format(rmse))

Training score: 0.2163641681056202
Test score: 0.23100926556834167
RMSE: 0.649656708246099


In [18]:
lr_model.coef_

array([-4.60774779e-06, -2.25323290e-02, -6.92850918e-03,  1.60597569e-08,
       -3.23079215e-06, -0.00000000e+00,  2.65227040e-03, -0.00000000e+00,
       -2.30761054e-06, -4.05571253e-03, -0.00000000e+00, -3.69583661e-03,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -7.35905701e-09,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -5.48025774e-04,  0.00000000e+00,
       -1.84123076e-03, -1.72670520e-03, -1.57963490e-04, -0.00000000e+00,
       -6.76617375e-04, -0.00000000e+00, -0.00000000e+00, -1.53994338e-03,
        2.99297122e-03,  7.86700697e-04,  3.31330074e-03, -4.91988114e-03,
        2.34189148e-04, -0.00000000e+00,  0.00000000e+00,  6.36163886e-03,
        1.93033240e-03, -0.00000000e+00, -0.00000000e+00, -1.51559374e-04,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        0.00000000e+00,  

In [19]:
rem_feature = list()
for i,j in enumerate(lr_model.coef_):
    if j == 0:
        rem_feature.append(i)
len(rem_feature)

34

In [20]:
X_train.columns[rem_feature]

Index(['age', 'weight_kg', 'club_contract_valid_until', 'weak_foot',
       'skill_moves', 'international_reputation', 'pace', 'shooting',
       'passing', 'dribbling', 'defending', 'physic', 'attacking_finishing',
       'skill_dribbling', 'skill_fk_accuracy', 'skill_long_passing',
       'power_shot_power', 'power_jumping', 'power_long_shots',
       'mentality_aggression', 'mentality_positioning', 'mentality_vision',
       'mentality_penalties', 'mentality_composure',
       'defending_marking_awareness', 'defending_standing_tackle',
       'defending_sliding_tackle', 'goalkeeping_diving',
       'goalkeeping_handling', 'goalkeeping_kicking',
       'goalkeeping_positioning', 'goalkeeping_reflexes', 'fwd', 'bck'],
      dtype='object')

In [21]:
rem_feature.remove(X_train.columns.get_loc("fwd"))
rem_feature.remove(X_train.columns.get_loc("bck"))

In [22]:
X_train.drop(labels=X_train.columns[rem_feature], inplace=True, axis=1)
X_test.drop(labels=X_test.columns[rem_feature], inplace=True, axis=1)

In [23]:
X_train.drop(labels="sofifa_id", inplace=True, axis=1)
X_test.drop(labels="sofifa_id", inplace=True, axis=1)

In [24]:
X_train

,overall,potential,value_eur,wage_eur,height_cm,club_team_id,club_jersey_number,nationality_id,release_clause_eur,attacking_crossing,...,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_stamina,power_strength,mentality_interceptions,fwd,mid,bck
12402,63,64,550000,2000,191,82,11,14,776000,29,...,74,59,58,47,65,74,22,4.704255,28.109957,41.346827
4007,71,71,1900000,16000,157,1882,10,57,4100000,65,...,85,94,74,93,70,58,29,30.060800,-18.514661,17.469516
7311,68,73,1600000,9000,180,101026,68,8,2100000,71,...,75,70,63,67,74,70,61,13.100581,-20.186657,-22.981187
9317,66,70,1100000,7000,185,110832,27,56,1600000,29,...,70,58,58,61,65,90,60,-21.030860,13.595584,-14.885177
2022,75,86,11500000,68000,190,241,6,18,23600000,46,...,78,54,70,56,64,79,75,-0.738147,-20.180241,-41.221377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15832,59,70,500000,2000,176,634,70,7,454000,54,...,85,76,44,73,46,42,18,-5.249792,29.374379,46.445734
6338,69,75,2000000,7000,185,110741,23,1,2600000,28,...,74,67,68,71,63,65,72,-31.113606,13.088246,-25.569496
1021,77,77,9500000,36000,177,219,17,18,14300000,77,...,81,74,77,75,88,69,72,46.302744,-55.942019,-45.355192
18757,50,69,120000,500,180,700,14,46,113000,44,...,62,63,53,66,46,52,45,-29.299965,30.707513,23.617897


In [25]:
sc = StandardScaler()

tr_col = list(X_train.columns)
te_col = list(X_test.columns)

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = pd.DataFrame(X_train, columns=tr_col)
X_test= pd.DataFrame(X_test, columns=te_col)

In [26]:
lr_model = RandomForestClassifier()
lr_model.fit(X_train, y_train)

print('Training score: {}'.format(lr_model.score(X_train, y_train)))
print('Test score: {}'.format(lr_model.score(X_test, y_test)))

y_pred = lr_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)

print('RMSE: {}'.format(rmse))

Training score: 1.0
Test score: 0.8728070175438597
RMSE: 0.5317803203448757
